In [143]:
import hashlib
import json
from bitarray import bitarray
from Cryptodome.Util import number
from merklelib import MerkleTree, beautify


# a sample hash function
# you can also omit it and the default hash function will be used
def split_in_128(value):
    unpadded = list(str(bin(int(value, 16))[2:]))
    padded = "".join(map(str, [0] * (256 - len(unpadded)) + unpadded))
    value1, value2 = padded[:128], padded[128:]
    return str(int(value1, 2)), str(int(value2, 2))

def printhash(value):
    print(f'''
    {'-' * 20}
    number: {value}
    hexdigest: {hex(value)}
    number size: {number.size(value)}
    128 pairs: {split_in_128(value)}
    ''')
    
def unpack256(value):
    return (value).to_bytes(32, byteorder='big')

def unpack512(value):
    return (value).to_bytes(64, byteorder='big')

def combine_128s(a, b):
    return (a << 128) + b
    
def leafhash(lhs, rhs):
    preimage = unpack256(lhs) +  unpack256(rhs)
    hex_hash = hashlib.sha256(preimage).hexdigest()
    hex_num = int(hex_hash,16)
    printhash(hex_num)
    return hex_num

def treehash(lhs, rhs):
    preimage = unpack512(lhs + rhs)
    hex_hash = hashlib.sha256(preimage).hexdigest()
    hex_num = int(hex_hash,16)
    printhash(hex_num)
    return hex_num

def format_barray(barray):
    barray_str = ''.join(barray)
    hashnum = int(barray_str, 2)
    printhash(hashnum)
    return hashnum

def digest_ecc_pairs(lhs, rhs):
    return unpack256(int(lhs)) +  unpack256(int(rhs))

def sha256(preimage):
    return hashlib.sha256(preimage).hexdigest()

def merkleTree(data):
    

In [145]:
voters = json.load(open("../voters.json", "r"))
voters_pk = [sha256(digest_ecc_pairs(voter.get('point_x'), voter.get('point_y'))) for voter in voters ]
tree = MerkleTree(voters_pk, sha256)
root = tree.merkle_root
beautify(tree)

f14565c7dd80ac11f353f8f0604f9efeb7f97b83b55234e0ca605f78525ea5f7
├── 04de61902c4200c769971ef2d3c96506cc921ebcf24105b512cefe8258a70e9c
│   ├── bdd7e02c1fe954e745f0398f21fde249f5e4454dd03668f035a0cd767dbc8c4b
│   │   ├── 4e85663296292c64e054f6f8da5ee54b2d79d8772c9882376c5bb94286aba10f
│   │   └── 8ea325a290710856f06108c7e46c9fd5bb0dad6a62327c38ee0ce666269d2659
│   └── 4ff33dc3c98d1e096f04041b51e0433626e677c00c2b32c0d074c6588eb9e465
│       ├── 53f0388af8ec86fa546b65aebe4863efb42e3e5bbd9caf8aa0443cbc36dc02c2
│       └── 9e60fa4697d7539e2a805d360c36e91bc271a3b15dd24b675b575789846d6d70
└── e9c53cfc6db50a755da26cce7c8e27e71bcd2841c3b879e3120707d6d525fb15
    ├── f7d6098f02016c2f5072b0d295c6bd14c946adee4bb4a6e32163a6c46b3e8c2d
    │   ├── d497b5967bb4990b1be3d1fed9fa8de7b35b224ee6986223458228e8d3a93922
    │   └── 54b3f57fb918fe8c8c78db603f8d400f6b2e114e4a97139f4b920e6af9c3a813
    └── 43490598e2eb4f0c0fb73ede5a049892a5d6873e5f340fc77f716aabbb36185b
        ├── 249bb9ce597f41be6bffdfed53aac1b

In [138]:
index = 1
voter = voters[index]
leaf_digest = ' '.join(split_in_128(voters_pk[index]))
root_digest = ' '.join(split_in_128(root))
pk = (voter.get('point_x'), voter.get('point_y'))
sk = voter.get('privatekey')
merkle_path = lambda x: ' '.join(list(str(bin((2**3) + x)))[3:][::-1])
path = merkle_path(index)
path_digest = ' '.join([ ' '.join(split_in_128(nodes)) for nodes in proof.hex_nodes])
proof = tree.get_proof(voters_pk[index])
proof.hex_nodes

ValueError: invalid literal for int() with base 16: b'\x16\xdc\x91?r\x9bB\x06\xd6*\xcf-\xc7\x825K;9\\\xe8\xa0\x08\x04<\xe82\x99\xd9h\xa6x\x84 4\xaf\xd8\x00\xf2\xd9uz\xc5\xfc\x17\xcev=\xb3\x98v\xee\x8d\xc5dcK~rMWcug3'

In [121]:
print(f'zokrates compute-witness -a {root_digest} {pk[0]} {pk[1]} {sk} {path} {leaf_digest} {path_digest}')

zokrates compute-witness -a 320704279825729679887093372602058514174 244544110317801483749573555211254801911 10340591486849500411597554648864878801016182183377116661129044131080489564292 14567100831092071334990535999461575549871942024819092424591682457990310750003 127701493367712430457058180578738495689 1 0 0 123747805351608569425204621146108226363 39540705688767375067091866303177216241 104372431967068795421462059261922043211 60447918172631368468733595847163093263 106271992542486917466246087527104529206 51707320939605894415659374384260965477 310734242443042663156803883748285097959 36954393248979672587276171610885061397


In [148]:
y = sha256(digest_ecc_pairs('19510076639286324373897420360096365090640882341900164882210954095146741092293', '8949781301061640769592652182028272879867087978866630439801287750411451384681'))
y

'92b34907ec85874ab6faadaa1fd32b3e86c1211c8fb9350a8bf13bd5caf1ff29'

In [154]:
sha256(y)

TypeError: Unicode-objects must be encoded before hashing

In [ ]:
print(f'{e} {f} {a} {b} 0 {c} {d}')

In [ ]:
str(split_in_128(tree_root))

In [ ]:
leafhash(9347197123955067502299347123172707107736742848980750471497860027091869058261, 17080122713037892253905384489626209271114931117541157439362422423664377917307)



In [ ]:
len('18108654401806847981431026731501438725632879398752240573157939653377929852131')

In [ ]:
leafhash(5273592313522545669763239369737548972936122350299813356657043169565692410214, 4438891100753198311257417672345655531574574993160995424947811462420457314719)


In [ ]:
leafhash(78500997768471140206194977071471243125152084370366652678202980359874644395177, 19467396431360704483580553484173217028284689943058940297520137226571781369538)


In [ ]:
type(78500997768471140206194977071471243125152084370366652678202980359874644395177)

In [155]:
'''
└─ 45193fd21ff8aa3309f72476546b40ef289e43e13b195557074c4c043f5fbc9e
   ├─ 38248f1953dce38d18c9c8008b59562275fa88398003941c3f318fb48840fe54
   │  ├─ a412baa4afe9d746f793d32ae25d8bff7f744cbf3565cf0af1560ee7c3c35026
   │  │  ├─ 92b34907ec85874ab6faadaa1fd32b3e86c1211c8fb9350a8bf13bd5caf1ff29
   │  │  └─ 5d18f5dbfb0469a7136a3f3f95f46b3b1dbf434e1f9d95574ed8d40f8a40d0f1
   │  └─ 411421fa567a310aec47f56965e2074c5dbfed12a0ec19aaa7f08f149addb6b4
   │     ├─ ad8dfd7d9441a9c44a796a319cf9a9cc80b056e06b7a3923cc20ce0c8ce1c4a9
   │     └─ 2b0a27e9cf8c514a1f01f7b905d6646b066b86a40f87c8fad4277c35cf85eec2
   └─ 17894b7aa5e52e75d1cbea9a822a05185acb0d882fe19eee11f9bd730ce3ece4
      ├─ 7bd12d142e707ca4e0fef03283fcb84428bc9d09c5094c3d695091a4bb6adc1f
      │  ├─ 369850511d58b5a40f57279d8ef1e18f84bfed2884ee463307a67077e4c75bf2
      │  └─ 9f762e459a5c3c6130ddaa04efa3d9aaf29424e73cb079b4f99a651332b8935a
      └─ 3bd093b55535e8ae2aac76beb51dacb2d09130e7a69678307795439fd2cfbb52
         ├─ 6760fd62081bbc67ae0320d61122e54871d026be124f41e11df2336077a93c39
         └─ 5bd9416d7cb4ff4560398e9d6b242fc3aaccea9c456038dcfb548a9ad8593945
'''

'\n└─ 45193fd21ff8aa3309f72476546b40ef289e43e13b195557074c4c043f5fbc9e\n   ├─ 38248f1953dce38d18c9c8008b59562275fa88398003941c3f318fb48840fe54\n   │  ├─ a412baa4afe9d746f793d32ae25d8bff7f744cbf3565cf0af1560ee7c3c35026\n   │  │  ├─ 92b34907ec85874ab6faadaa1fd32b3e86c1211c8fb9350a8bf13bd5caf1ff29\n   │  │  └─ 5d18f5dbfb0469a7136a3f3f95f46b3b1dbf434e1f9d95574ed8d40f8a40d0f1\n   │  └─ 411421fa567a310aec47f56965e2074c5dbfed12a0ec19aaa7f08f149addb6b4\n   │     ├─ ad8dfd7d9441a9c44a796a319cf9a9cc80b056e06b7a3923cc20ce0c8ce1c4a9\n   │     └─ 2b0a27e9cf8c514a1f01f7b905d6646b066b86a40f87c8fad4277c35cf85eec2\n   └─ 17894b7aa5e52e75d1cbea9a822a05185acb0d882fe19eee11f9bd730ce3ece4\n      ├─ 7bd12d142e707ca4e0fef03283fcb84428bc9d09c5094c3d695091a4bb6adc1f\n      │  ├─ 369850511d58b5a40f57279d8ef1e18f84bfed2884ee463307a67077e4c75bf2\n      │  └─ 9f762e459a5c3c6130ddaa04efa3d9aaf29424e73cb079b4f99a651332b8935a\n      └─ 3bd093b55535e8ae2aac76beb51dacb2d09130e7a69678307795439fd2cfbb52\n         ├─ 6760